# This notebook provide the experience of creating a model using training job and deploying an endpoint - Boto3 / SageMaker V2 / SageMaker V3

Note everything that can be done in V2 will be available in V3. We are adding code-gen part so customer can program in object modeling along with other constructs in V2.

## Boto3 experience

Create a xgboost model using sagemaker create training job using only boto3 and deploy the model to an endpoint

In [ ]:
import time
import boto3

client = boto3.client('sagemaker')
response = client.create_training_job(
    TrainingJobName='xgboost-iris',
    HyperParameters={
        'objective': 'multi:softmax',
        'num_class': '3',
        'num_round': '10',
        'eval_metric': 'merror'
    },
    AlgorithmSpecification={
        'TrainingImage': '811284229777.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
        'TrainingInputMode': 'File'
    },
    RoleArn='arn:aws:iam::xxxxxxxxxxxx:role/service-role/AmazonSageMaker-ExecutionRole-20181029T134365',
    InputDataConfig=[
        {
            'ChannelName': 'train',
            'DataSource': {
                'S3DataSource': {
                    'S3DataType': 'S3Prefix',
                    'S3Uri': 's3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/train',
                    'S3DataDistributionType': 'FullyReplicated'
                }
            },
            'CompressionType': 'None',
            'RecordWrapperType': 'None'
        }
    ],
    OutputDataConfig={
        'S3OutputPath': 's3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/output'
    },
    ResourceConfig={
        'InstanceType': 'ml.m4.xlarge',
        'InstanceCount': 1,
        'VolumeSizeInGB': 30
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 600
    }
)

# check for the status of the training job creation
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Training job created successfully')
else:
    print('Failed to create training job')
    exit(1)

# describe the status of the training job, if it is completed, deploy the model else wait and loop in completion
condition = True

while condition:
    response = client.describe_training_job(TrainingJobName='xgboost-iris')
    if response['TrainingJobStatus'] == 'Completed':
        condition = False
    else:
        # wait for 5 minutes before checking the status again
        time.sleep(300) 

    print('Training job status: {}'.format(response['TrainingJobStatus']))

# create a model from the training job
response = client.create_model(
    ModelName='xgboost-iris',
    PrimaryContainer={
        'Image': '811284229777.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
        'ModelDataUrl': response['ModelArtifacts']['S3ModelArtifacts'] # Note, there are lot more in converting model from training job to deployable model (which ModelBuilder covers). I am passing the complexity for now.
    },
    ExecutionRoleArn='arn:aws:iam::xxxxxxxxxxxx:role/service-role/AmazonSageMaker-ExecutionRole-20181029T134365'
)

# check for the status of the model creation
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Model created successfully')
else:
    print('Failed to create model')
    exit(1)

# create an endpoint configuration
response = client.create_endpoint_config(
    EndpointConfigName='xgboost-iris',
    ProductionVariants=[
        {
            'VariantName': 'xgboost-iris',
            'ModelName': 'xgboost-iris',
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m4.xlarge'
        }
    ]
)

# check for the status of the endpoint configuration creation
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Endpoint configuration created successfully')
else:
    print('Failed to create endpoint configuration')
    exit(1)

# create an endpoint
response = client.create_endpoint(
    EndpointName='xgboost-iris',
    EndpointConfigName='xgboost-iris'
)

# check for the status of the endpoint creation
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print('Endpoint created successfully')
else:
    print('Failed to create endpoint')
    exit(1)

# describe the status of the endpoint, if it is in service, the endpoint is ready for use
condition = True

while condition:
    response = client.describe_endpoint(EndpointName='xgboost-iris')
    if response['EndpointStatus'] == 'InService':
        condition = False
    else:
        # wait for 5 minutes before checking the status again
        time.sleep(300)

    print('Endpoint status: {}'.format(response['EndpointStatus']))

print('Endpoint is ready for use')

# note, we are using different client to call runtime client
runtime_client = boto3.client('sagemaker_runtime')
response = runtime_client.invoke_endpoint(
    EndpointName='xgboost-iris', # and we pass the endpoint name here
    Body=b'{"data": [5.1, 3.5, 1.4, 0.2]}',
    ContentType='application/json',
    Accept='application/json'
)

# SageMaker v2

Note the below example looks simple for xgboost usecase, because I picked an example that is supported in the estimator. 

Example TrainingImageConfig and ShadowVariants are not supported in the below example

In [ ]:
# create the experience using sagemaker v2 using estimator and deploy

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

# create a sagemaker estimator
container = get_image_uri(sess.boto_region_name, 'xgboost')

# Note the below example looks simple for xgboost usecase, because I picked an example that is supported in the estimator. 
# e.g
# TrainingImageConfig = {
#    'TrainingRepositoryAccessMode': 'Platform'|'Vpc',
#            'TrainingRepositoryAuthConfig': {
#                'TrainingRepositoryCredentialsProviderArn': 'string'
#            }
# is not supported in estimator

xgb = sagemaker.estimator.Estimator(container,
                                    role,
                                    train_instance_count=1,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/output',
                                    sagemaker_session=sess)

xgb.set_hyperparameters(objective='multi:softmax',
                        num_class=3,
                        num_round=10,
                        eval_metric='merror')

# set the training data location
train_input = sagemaker.s3_input(s3_data='s3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/train', content_type='csv')

xgb.fit({'train': train_input})

# deploy the model to an endpoint
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
xgb_predictor.predict('{"data": [5.1, 3.5, 1.4, 0.2]}')

# SageMaker V3 code gen sample (note estimator will continue to work in v3)

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker import TrainingJob, Model, Endpoint, EndpointConfig

sess = sagemaker.Session()

# create a training job
training_job = TrainingJob.create(
    training_job_name='xgboost-iris',
    hyper_parameters = HyperParameters(
        objective='multi:softmax',
        num_class='3',
        num_round='10',
        eval_metric='merror'
    ),
    algorithm_specification = AlgorithmSpecification(
        training_image='811284229777.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
        training_input_mode='File'
    ),
    role_arn='arn:aws:iam::xxxxxxxxxxxx:role/service-role/AmazonSageMaker-ExecutionRole-20181029T134365',
    input_data_config = [
        Channel(
            channel_name='train',
            data_source=DataSource(
                s3_data_source=S3DataSource(
                    s3_data_type='S3Prefix',
                    s3_uri='s3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/train',
                    s3_data_distribution_type='FullyReplicated'
                )
            ),
            compression_type='None',
            record_wrapper_type='None'
        )
    ],
    output_data_config = OutputDataConfig(
        s3_output_path='s3://sagemaker-us-west-2-xxxxxxxxxxxx/iris/output'
    ),
    resource_config = ResourceConfig(
        instance_type='ml.m4.xlarge',
        instance_count=1,
        volume_size_in_gb=30
    ),
    stopping_condition = StoppingCondition(
        max_runtime_in_seconds=600
    ),
    session=sess, # we can improve passing session and region automatically if it is running in studio notebook - customers can pass on this.
    region=sess.boto_region_name  # we can improve passing session and region automatically if it is running in studio notebook - customers can pass on this.
)

# wait for the training job to complete
training_job.wait() # wait is an object method that will wait for the training job to complete and only supported in empheral workload

'''
or customer can do 

while training_job.training_job_status != 'Completed':
    time.sleep(300)
    training_job.refresh()
'''

model = Model.create(
    model_name='xgboost-iris',
    primary_container=Container(
        image='811284229777.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
        model_data_url=training_job.model_artifacts.s3_model_artifacts # here we are getting model data from the training job 
    ),
    execution_role_arn='arn:aws:iam::xxxxxxxxxxxx:role/service-role/AmazonSageMaker-ExecutionRole-20181029T134365',
)
'''
if the model is already created then 
we can use Model.get() to retrive the model
'''

endpoint_config = EndpointConfig.create(
    endpoint_config_name='xgboost-iris',
    production_variants=[
        ProductionVariant(
            variant_name='xgboost-iris',
            model_name=model, # note we can chain it to get the name automatically
            initial_instance_count=1,
            instance_type='ml.m4.xlarge'
        )
    ],
)

endpoint = Endpoint.create(
    endpoint_name='xgboost-iris',
    endpoint_config_name=endpoint_config # note we can chain it to get the name automatically
)

# wait for the endpoint to be in service
endpoint.wait_for_status('InService')

endpoint.invoke(body=b'{"data": [5.1, 3.5, 1.4, 0.2]}', # note the invoke is part of the endpoint object. 
                content_type='application/json',
                accept='application/json')

endpoint.delete()

# Do Inference experiment for shadow deployment
InferenceExperiment = InferenceExperiment.create(
    name='xgboost-iris-shadow',
    type='ShadowMode',
    schedule = Schedule(
        start_time='2021-01-01T00:00:00Z',
        end_time='2021-01-02T00:00:00Z',
    )
    endpoint_name='xgboost-iris',
    model_variants=[
        ModelVariant(
            variant_name='xgboost-iris',
            model_name='xgboost-iris',
            initial_instance_count=1,
            instance_type='ml.m4.xlarge'
        )
    ],
    shadow_mode_config=ShadowModeConfig(
        source_model_variant_name='xgboost-iris',
        shadow_model_variants=[
            ShadowModelVariant(
                shadow_model_name='xgboost-iris-shadow',
                sampling_percentage=10
            )
        ]
    )
)